In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

#import src
import Cleanup
import Transformation
import FileProcessing

In [2]:
#define the objects to be called later. 
cleanup=Cleanup.Cleanup()
transform=Transformation.Transformation()
fileProcessing=FileProcessing.FileProcessing()

Read the File from Data folder and remove all duplicates. 

In [3]:
file_name='Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data'
ngsim=fileProcessing.read_input(file_name)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Data File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


Remove any Duplicate records from the original File. 

In [4]:
cleanup.remove_dups(ngsim)

704000 duplicate values have been removed


,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,D_Zone,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location
0,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101
2,2224,6548,1902,1113437421700,41.429,472.901,6042814.264,2133542.012,14.3,6.9,...,NaN,NaN,NaN,NaN,NaN,2208,2211,53.34,2.01,i-80
3,2127,6459,567,1118847624800,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,...,NaN,NaN,NaN,NaN,NaN,2124,2132,48.92,1.30,us-101
4,1033,4827,592,1118848324700,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,...,NaN,NaN,NaN,NaN,NaN,1029,1040,38.81,0.92,us-101
6,1890,9157,628,1118849672700,53.514,817.521,6451655.238,1872800.663,24.0,8.5,...,NaN,NaN,NaN,NaN,NaN,1882,1897,102.65,2.27,us-101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11850521,1355,5622,938,1113438127100,18.275,1464.336,6042648.149,2134520.896,12.3,6.8,...,NaN,NaN,NaN,NaN,NaN,1351,1361,26.91,15.92,i-80
11850522,1474,5688,401,1113438133700,5.509,560.412,6042767.840,2133624.549,17.8,7.9,...,NaN,NaN,NaN,NaN,NaN,1468,1480,74.49,1.73,i-80
11850523,398,2368,654,1113437801700,18.903,635.841,6042771.741,2133701.076,15.2,8.5,...,NaN,NaN,NaN,NaN,NaN,385,406,90.95,3.77,i-80
11850524,599,875,577,1113436854400,77.094,621.358,6042831.282,2133693.854,15.3,6.4,...,NaN,NaN,NaN,NaN,NaN,0,611,0.00,0.00,i-80


Feet to Metres:

Below Four parametrs consider distance in Feet and they are converted to metre. Speed to Metre/ second from Feet/second and acceleration from feet/second Square to Metre/Second Square. 

Drop the columns we wont use in the model.
Add Relative time which is the time from the first timframe in the dataset. This is calculated by using the minimum value ofGlobal time(absolute time from 1970)

In [5]:
ngsim=transform.convert_feet_to_metre(ngsim)

In [6]:
ngsim.shape

(11146526, 25)

In [7]:
ngsim=transform.drop_not_required_columns(ngsim)

Create Placeholders for the user defined columns we will be populating later, to be used in the model. 
Map the Vehicle classes to their english equivalent. 
Filter the original file into US-101 and I-80 to perform and populate the required fields later. These files contain same values of Vehicle ID used for multiple vehicles based on the highway. 



In [8]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim=transform.create_column_placeholders(ngsim)

Split data into US-101 and I-80 specific dataframes.

In [9]:
filtered_ngsim_U,filtered_ngsim_I = transform.bifurcate_highways(ngsim)

Create Sets to use and create the L-F Pairs. 

In [10]:
v_Class_M_U,v_Class_C_U,v_Class_HV_U = transform.class_vehicle_sets(filtered_ngsim_U)
v_Class_M_I,v_Class_C_I,v_Class_HV_I = transform.class_vehicle_sets(filtered_ngsim_I)


Add the Preceding Vehicle Class to the new previous class column

In [11]:
filtered_ngsim_U=transform.preceding_vehicle_class(filtered_ngsim_U,v_Class_M_U,v_Class_C_U,v_Class_HV_U)
filtered_ngsim_I=transform.preceding_vehicle_class(filtered_ngsim_I,v_Class_M_I,v_Class_C_I,v_Class_HV_I)

Map the Vehicle Length of the previous vehicle into the row. 

In [12]:
filtered_ngsim_U=transform.preceding_vehicle_length(filtered_ngsim_U)
filtered_ngsim_I=transform.preceding_vehicle_length(filtered_ngsim_I)

Convert the Front to Front Space Headway to Back Bumper (preceding Vehicle) to front Bumper(of Following ) Headway. This will give the actual distance between the vehicle ignoring the vehicle length.
Find the time it would take for the Following vehicle to reach the rear Bumper of the Preceding Vehicle. 

In [13]:
filtered_ngsim_U= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_U)
filtered_ngsim_I= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_I)

Vehicle Combination of the Preceding and the Following Vehicle. 

In [14]:
filtered_ngsim_U=transform.map_preceding_vehicle_details(filtered_ngsim_U)
filtered_ngsim_I=transform.map_preceding_vehicle_details(filtered_ngsim_I)

us-101::621 cars dont change lanes
us-101::2226 cars Change lanes
i-80::817 cars dont change lanes
i-80::2184 cars Change lanes


In [15]:
filtered_ngsim_U=transform.map_pairs(filtered_ngsim_U)
filtered_ngsim_I=transform.map_pairs(filtered_ngsim_I)

filtered_ngsim_U=cleanup.filter_records_for_model(filtered_ngsim_U)
filtered_ngsim_I=cleanup.filter_records_for_model(filtered_ngsim_I)



dataset before Row Removal(4098933, 33)
us-101: 154876 vehicles first and Last 5 seconds removed from Lane 1 due to lane changing
us-101: 194882 vehicles first and Last 5 seconds removed from Lane 2 due to lane changing
us-101: 195007 vehicles first and Last 5 seconds removed from Lane 3 due to lane changing
us-101: 530853 have multiple lengths and classes for same Vehicle ID
us-101: 780619 have total car following less than 30 seconds
us-101: 376267 have time Headway less than 5 seconds
us-101: 1598920 have Lane ID as 7 or 8 which are the Ramps
us-101: 2629503 rows removed using above criterias
dataset before Row Removal(4566387, 33)
i-80: 153820 vehicles first and Last 5 seconds removed from Lane 1 due to lane changing
i-80: 157225 vehicles first and Last 5 seconds removed from Lane 2 due to lane changing
i-80: 163719 vehicles first and Last 5 seconds removed from Lane 3 due to lane changing
i-80: 830630 have multiple lengths and classes for same Vehicle ID
i-80: 940574 have total ca

In [16]:
filtered_ngsim_I.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Following', 'Space_Headway', 'Time_Headway', 'Location',
       'Preceding_Vehicle_Class', 'Rear_to_Front_Space_Headway',
       'Front_To_Rear_Time_Headway', 'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length',
       'lane_changes', 'Prec_Vehicle_ID', 'preceding_Vehicle_Velocity',
       'preceding_Vehicle_Acceleration', 'preceding_car_lane_changes',
       'preceding_Local_Y', 'preceding_v_Class', 'pair_Time_Duration',
       'total_pair_duration'],
      dtype='object')

In [17]:
#dsasas

In [18]:
filtered_ngsim_U.drop(columns=['pair_Time_Duration'],axis=1,inplace=True)
filtered_ngsim_I.drop(columns=['pair_Time_Duration'],axis=1,inplace=True)
filtered_ngsim_U=transform.remap_pair_time(filtered_ngsim_U)
filtered_ngsim_I=transform.remap_pair_time(filtered_ngsim_I)

In [19]:
filtered_ngsim_I['diference'] = filtered_ngsim_I['total_pair_duration'] - filtered_ngsim_I['total_pair_dur'] 

In [20]:
filtered_ngsim_I[filtered_ngsim_I['diference'] >10]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,total_pair_duration,pair_Time_Duration,total_pair_dur,diference
81,11,143,1113433149200,4.838090,36.426343,5.27304,2,2.688336,3.413760,2,...,1,2.660904,-0.054864,False,48.995381,2.0,83.8,0.0,61.9,21.9
84,11,144,1113433149300,4.856988,36.732667,5.27304,2,3.374136,3.413760,2,...,1,2.667000,0.112776,False,49.260252,2.0,83.8,0.1,61.9,21.9
85,11,145,1113433149400,4.863084,37.087759,5.27304,2,3.828288,2.627376,2,...,1,2.706624,0.612648,False,49.524209,2.0,83.8,0.2,61.9,21.9
87,11,146,1113433149500,4.858817,37.516918,5.27304,2,3.736848,-3.413760,2,...,1,2.801112,1.252728,False,49.793652,2.0,83.8,0.3,61.9,21.9
89,11,147,1113433149600,4.801819,37.844273,5.27304,2,3.023616,-3.413760,2,...,1,2.935224,1.557528,False,50.077726,2.0,83.8,0.4,61.9,21.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870458,2483,11168,1113438681700,9.555785,463.863233,3.90144,2,3.669792,-1.575816,3,...,2479,5.458968,0.000000,False,474.434002,2.0,199.1,99.5,99.9,99.2
870459,2483,11169,1113438681800,9.562490,464.204304,3.90144,2,3.651504,1.179576,3,...,2479,5.458968,0.000000,False,474.979898,2.0,199.1,99.6,99.9,99.2
870460,2483,11170,1113438681900,9.553956,464.563358,3.90144,2,3.828288,2.831592,3,...,2479,5.458968,0.000000,False,475.525795,2.0,199.1,99.7,99.9,99.2
870461,2483,11171,1113438682000,9.539326,464.962037,3.90144,2,4.066032,2.356104,3,...,2479,5.458968,0.000000,False,476.071692,2.0,199.1,99.8,99.9,99.2


In [21]:
filtered_ngsim_I[filtered_ngsim_I['total_pair_dur']<30]



,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,total_pair_duration,pair_Time_Duration,total_pair_dur,diference
1076,116,467,1113433181600,5.085588,30.611369,4.84632,2,3.096768,1.304544,2,...,97,2.462784,3.413760,False,45.663917,2.0,42.3,0.0,29.4,12.9
1081,116,468,1113433181700,5.085893,30.933542,4.84632,2,3.188208,0.408432,2,...,97,2.883408,3.413760,False,45.932141,2.0,42.3,0.1,29.4,12.9
1092,116,469,1113433181800,5.085893,31.258154,4.84632,2,3.209544,-0.070104,2,...,97,3.243072,2.822448,False,46.240903,2.0,42.3,0.2,29.4,12.9
1097,116,470,1113433181900,5.085893,31.579109,4.84632,2,3.203448,-0.070104,2,...,97,3.386328,0.057912,False,46.597519,2.0,42.3,0.3,29.4,12.9
1102,116,471,1113433182000,5.085893,31.898539,4.84632,2,3.200400,0.000000,2,...,97,3.297936,-2.139696,False,46.952002,2.0,42.3,0.4,29.4,12.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870338,2480,11016,1113438666500,8.482279,436.731461,4.35864,2,2.039112,0.064008,3,...,2483,0.000000,0.000000,True,446.391487,2.0,80.7,24.0,24.4,56.3
870339,2480,11017,1113438666600,8.481365,436.935372,4.35864,2,2.039112,0.070104,3,...,2483,0.000000,0.000000,True,446.391487,2.0,80.7,24.1,24.4,56.3
870342,2480,11018,1113438666700,8.480450,437.143246,4.35864,2,2.017776,-0.463296,3,...,2483,0.000000,0.000000,True,446.391487,2.0,80.7,24.2,24.4,56.3
870344,2480,11019,1113438666800,8.479841,437.347766,4.35864,2,1.929384,-1.274064,3,...,2483,0.000000,0.000000,True,446.391487,2.0,80.7,24.3,24.4,56.3



1. Velocity difference of the Lead minus Preceding Vehicle.
2. Acceleration difference of the Lead minus Preceding Vehicle.
3. Find the Pair details of the Lead_following 
4. Find the cumulative time the Pair runs within the highway as a pair. 

In [23]:
filtered_ngsim= fileProcessing.merge_files(filtered_ngsim_U , filtered_ngsim_I)
filtered_ngsim= filtered_ngsim[filtered_ngsim['total_pair_dur']>=30]
filtered_ngsim=cleanup.remove_dup_pairs(filtered_ngsim)

 Merged Record Count:2339893, df1:1469430, df2:870463
0 rows removed for Dups. Count Before removal: 2076597, Count After Removal:2076597


In [28]:
x=filtered_ngsim.groupby(['Vehicle_combination','L-F_Pair'])['total_pair_duration'].max().sort_values(ascending=False)

In [29]:
x_hv_car=x[x.index.get_level_values('Vehicle_combination').isin(['Heavy Vehicle-Car'])]
hv_car_pair= x_hv_car[x_hv_car>60].tail(3).index.get_level_values('L-F_Pair').values
hv_car_pair


array(['3193-3209', '3084-3094', '551-560'], dtype=object)

In [30]:
x_car_car=x[x.index.get_level_values('Vehicle_combination').isin(['Car-Car'])]
Car_car_pair= x_car_car[x_car_car>60].tail(3).index.get_level_values('L-F_Pair').values
Car_car_pair


array(['390-394', '249-255', '439-444'], dtype=object)

In [51]:
x_car_hv=x[x.index.get_level_values('Vehicle_combination').isin(['Car-Heavy Vehicle'])]
car_hv_pair= x_car_hv[x_car_hv>60].tail(3).index.get_level_values('L-F_Pair').values
car_hv_pair


array(['2322-2330', '2785-2804', '2725-2717'], dtype=object)

In [58]:
trajectory_pairs=filtered_ngsim[filtered_ngsim['L-F_Pair'].isin(car_hv_pair )|filtered_ngsim['L-F_Pair'].isin(Car_car_pair)|filtered_ngsim['L-F_Pair'].isin(hv_car_pair)]

In [59]:
trajectory_pairs

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,total_pair_duration,pair_Time_Duration,total_pair_dur,diference
855679,2330,8430,1118848685000,8.434121,75.145087,6.70560,3,9.140952,0.000000,3,...,2322,7.604760,0.000000,True,91.089175,2.0,66.8,0.0,56.7,NaN
855745,2330,8431,1118848685100,8.458200,76.059182,6.70560,3,9.140952,0.000000,3,...,2322,7.601712,-0.012192,True,91.849651,2.0,66.8,0.1,56.7,NaN
855759,2330,8432,1118848685200,8.482279,76.973278,6.70560,3,9.140952,0.000000,3,...,2322,7.601712,-0.018288,True,92.609822,2.0,66.8,0.2,56.7,NaN
855814,2330,8433,1118848685300,8.506358,77.887373,6.70560,3,9.140952,0.000000,3,...,2322,7.607808,0.112776,True,93.369079,2.0,66.8,0.3,56.7,NaN
855879,2330,8434,1118848685400,8.530133,78.800858,6.70560,3,9.147048,0.121920,3,...,2322,7.635240,0.371856,True,94.128641,2.0,66.8,0.4,56.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613863,394,2681,1113437833000,9.216238,437.141112,4.35864,2,8.790432,0.719328,3,...,390,8.177784,-0.036576,True,457.575514,2.0,60.1,49.7,50.1,10.0
613903,394,2682,1113437833100,9.169603,438.018326,4.35864,2,8.884920,1.435608,3,...,390,8.174736,-0.094488,True,458.393902,2.0,60.1,49.8,50.1,10.0
613946,394,2683,1113437833200,9.143695,438.915048,4.35864,2,9.006840,1.170432,3,...,390,8.168640,0.000000,True,459.210156,2.0,60.1,49.9,50.1,10.0
613975,394,2684,1113437833300,9.140038,439.826095,4.35864,2,9.095232,0.521208,3,...,390,8.168640,0.000000,True,460.027020,2.0,60.1,50.0,50.1,10.0


In [60]:
filtered_ngsim = filtered_ngsim.drop(trajectory_pairs.index,axis=0)

In [61]:
fileProcessing.export_file(filtered_ngsim,'Cleaned_NGSIM_Data')
fileProcessing.export_file(trajectory_pairs,'Plot_Pairs_NGSIM_Data')
filtered_ngsim.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Following', 'Space_Headway', 'Time_Headway', 'Location',
       'Preceding_Vehicle_Class', 'Rear_to_Front_Space_Headway',
       'Front_To_Rear_Time_Headway', 'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length',
       'lane_changes', 'Prec_Vehicle_ID', 'preceding_Vehicle_Velocity',
       'preceding_Vehicle_Acceleration', 'preceding_car_lane_changes',
       'preceding_Local_Y', 'preceding_v_Class', 'total_pair_duration',
       'pair_Time_Duration', 'total_pair_dur', 'diference'],
      dtype='object')

In [62]:
print(f"{filtered_ngsim[(filtered_ngsim['preceding_car_lane_changes'] == True) ]['Vehicle_ID'].unique().size} cars Change lanes")
print(f"{filtered_ngsim[(filtered_ngsim['preceding_car_lane_changes'] == False) ]['Vehicle_ID'].unique().size} cars DonT")

1924 cars Change lanes
643 cars DonT


In [63]:
filtered_ngsim['L-F_Pair'].unique().shape

(3655,)

In [64]:
filtered_ngsim[((filtered_ngsim['Space_Headway'] == 0 ) & (filtered_ngsim['Preceding'] > 0 )) ]


,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,total_pair_duration,pair_Time_Duration,total_pair_dur,diference
398182,2964,8731,1118847852000,1.483766,94.405399,5.02920,2,2.618232,-3.413760,1,...,2956,1.411224,1.298448,False,94.406923,2.0,89.6,4.8,57.7,NaN
489350,2491,7916,1113437558500,5.230673,182.539843,4.66344,2,3.453384,0.036576,2,...,2486,3.200400,0.000000,False,182.540148,2.0,99.8,24.3,79.6,20.2
